# Grouping, Pivoting, and Merging

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.precision', 2)

In [2]:
dat = pd.read_csv('starmine.csv', parse_dates=['date'])
# dat = dat.set_index(['date', 'symbol'], verify_integrity=True).sort_index()

sectors = 'Durbl Enrgy HiTec'.split(' ')
dates = ['1995-01-31', '1995-02-28']
cols = ['date', 'symbol', 'name', 'sector', 'size', 
        'smi', 'liq', 'ret_0_1_m', 'cap_usd']
dat = dat[cols].query('sector in @sectors and date in @dates').reset_index(drop=True)

In [3]:
sp5 = pd.read_csv('sp500.csv', parse_dates=['date'], index_col=['date']).sort_index()

## Split, Apply, Combine

Hadley Wickham  
**The split-apply-combine strategy for data analysis.**  
_Journal of Statistical Software_, vol. 40, no. 1, pp. 1–29, 2011  

In [4]:
grp = dat.groupby('sector')

## Split

Conceptually, allows iteration over the split `DataFrame`

In [5]:
sector_name, sector_df = iter(grp).send(None)

# Same as this
# for sector_name, grp_df in grp:
#     ...

sector_name

'Durbl'

In [6]:
sector_df.iloc[0:3, 0:3]

,date,symbol,name
4,1995-01-31,3FDMLQ,Federal-mogul Corp
28,1995-01-31,AICOQ,Amcast Industrial Corp
29,1995-01-31,AIHI,Automotive Inds Hldgs -cl A


## Apply

Default applies to all non-numeric columns.

In [7]:
# Only select 5 columns of result
grp.mean().iloc[:, 0:5]

,size,smi,liq,ret_0_1_m,cap_usd
sector,,,,,
Durbl,-0.14,47.60,0.08,0.01,2.06e+09
Enrgy,-0.06,31.30,0.13,0.05,3.87e+09
HiTec,-0.51,61.95,0.27,0.06,1.48e+09


# Apply

Apply to a single column.

In [8]:
# Returns a Series
grp['smi'].mean()

sector
Durbl    47.60
Enrgy    31.30
HiTec    61.95
Name: smi, dtype: float64

In [9]:
# Returns a DataFrame
grp[['smi']].mean()

,smi
sector,
Durbl,47.60
Enrgy,31.30
HiTec,61.95


# Apply

Apply the same function to multiple *selected* columns.

In [10]:
grp[['smi', 'cap_usd']].mean()

,smi,cap_usd
sector,,
Durbl,47.60,2.06e+09
Enrgy,31.30,3.87e+09
HiTec,61.95,1.48e+09


# Apply

Apply different functions to a single column and give the
result `DataFrame` custom names.

Use the `aggregate` or `agg` method.

In [11]:
grp['smi'].agg({
    'avg': lambda x: x.mean(),
    'median': lambda x: x.median(),
    'q75': lambda x: x.dropna().quantile(0.75)
})

/Users/daniel/anaconda3/envs/pydata/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


,avg,median,q75
sector,,,
Durbl,47.60,44.0,69.0
Enrgy,31.30,27.0,44.0
HiTec,61.95,65.0,88.0


# Apply

Apply different functions to a single column. Results have the same
names as the functions.

In [12]:
grp['smi'].agg(['mean', 'median'])

,mean,median
sector,,
Durbl,47.60,44.0
Enrgy,31.30,27.0
HiTec,61.95,65.0


# Apply

Apply different functions to different columns.

Again, use the `aggregate` or `agg` method.

In [13]:
grp.agg({
    'smi': lambda x: x.median(),
    'cap_usd': 'mean'
})

,smi,cap_usd
sector,,
Durbl,44.0,2.06e+09
Enrgy,27.0,3.87e+09
HiTec,65.0,1.48e+09


## Grouping by Multiple Variables

Same idea as before, except our results now have a MultiIndex.

In [14]:
grp2 = dat.groupby(['sector', 'date'])
grp2[['size', 'smi']].mean()

size    smi
sector date                   
Durbl  1995-01-31 -0.18  50.58
       1995-02-28 -0.10  44.44
Enrgy  1995-01-31 -0.05  28.49
       1995-02-28 -0.07  34.17
HiTec  1995-01-31 -0.51  61.98
       1995-02-28 -0.50  61.92

# Flexible Apply

Use `apply` to operate on each grouped subset of the `DataFrame`

In [15]:
grp3 = dat.groupby('sector')
grp3.apply(lambda df: pd.Series(df.shape, index=['nrow', 'ncol']))

,nrow,ncol
sector,,
Durbl,131,9
Enrgy,164,9
HiTec,672,9
